In [1]:
import dask.dataframe as dd

import os

In [2]:
os.getcwd()
os.chdir('/')


In [3]:
os.getcwd()

'/'

In [4]:
#according to the documentation this file is tab delimited format
names = ['Protein_accession',
    'Sequence_MD5_',
    'Sequence_length',
    'Analysis',
    'Signature_accession',
    'Signature_description',
    'Start_location',
    'Stop_location',
    'Score',
    'Status',
    'Date',
    'InterPro_accession',
    'InterPro_description',
    'GO_annotations',
    'Pathways_annotations']
dtypes = {
    'Protein_accession':'string',
    'Sequence_MD5_': 'string',
    'Sequence_length': int,
    'Analysis': 'string',
    'Signature_accession': 'string',
    'Signature_description': 'string',
    'Start_location': int,
    'Stop_location': int,
    'Score': 'object',
    'Status': 'string',
    'Date': 'string',
    'InterPro_accession':'string',
    'InterPro_description':'string',
    'GO_annotations':'string',
    'Pathways_annotations':'string'
}


ddf = dd.read_table('/data/dataprocessing/interproscan/all_bacilli.tsv', sep='\t', header=None, names=names, dtype=dtypes).get_partition(0)

The TSV format presents the match data in columns as follows:

    Protein accession (e.g. P51587)
    Sequence MD5 digest (e.g. 14086411a2cdf1c4cba63020e1622579)
    Sequence length (e.g. 3418)
    Analysis (e.g. Pfam / PRINTS / Gene3D)
    Signature accession (e.g. PF09103 / G3DSA:2.40.50.140)
    Signature description (e.g. BRCA2 repeat profile)
    Start location
    Stop location
    Score - is the e-value (or score) of the match reported by member database method (e.g. 3.1E-52)
    Status - is the status of the match (T: true)
    Date - is the date of the run
    InterPro annotations - accession (e.g. IPR002093)
    InterPro annotations - description (e.g. BRCA2 repeat)
    (GO annotations (e.g. GO:0005515) - optional column; only displayed if –goterms option is switched on)
    (Pathways annotations (e.g. REACT_71) - optional column; only displayed if –pathways option is switched on)


In [5]:
ddf.columns

Index(['Protein_accession', 'Sequence_MD5_', 'Sequence_length', 'Analysis',
       'Signature_accession', 'Signature_description', 'Start_location',
       'Stop_location', 'Score', 'Status', 'Date', 'InterPro_accession',
       'InterPro_description', 'GO_annotations', 'Pathways_annotations'],
      dtype='object')

# 1. How many distinct protein annotations are found in the dataset? I.e. how many distanct intoPro numbers are there

In [6]:
# -1 to remove the nan value in this column
len(ddf.InterPro_accession.unique().compute()) -1

3208

# 2. How many annotations does a protein have on average?

In [7]:
ddf2 = ddf[['Protein_accession', 'InterPro_description']]
ddf2 = ddf2[ddf2 != '-']

ddf2.groupby('Protein_accession')['InterPro_description'].nunique().mean().compute()

2.924808644754615

# 3. What is the most common GO Term found?

In [28]:
from collections import Counter
ddf3 = ddf.GO_annotations
ddf3 = ddf3.dropna()
ddf3 = ddf3[ddf3 != '-']
summation = ddf3.str.split('|').sum().compute()
test = Counter(summation)
sorted(test.items(), key=lambda x:x[1], reverse=True)[:10]

[('GO:0005524', 526),
 ('GO:0003677', 431),
 ('GO:0006355', 408),
 ('GO:0016020', 394),
 ('GO:0016021', 316),
 ('GO:0055085', 213),
 ('GO:0003700', 212),
 ('GO:0003824', 204),
 ('GO:0016491', 182),
 ('GO:0005515', 164)]

# 4. What is the average size of an InterPRO feature found in the dataset?

In [37]:
ddf4 = ddf[['InterPro_accession', 'Sequence_length']]
ddf4 = ddf4[ddf4 != '-']
ddf4 = ddf4.dropna()
ddf4.groupby('InterPro_accession').sum().mean().compute()

Sequence_length    1358.930486
dtype: float64

# 5. What is the top 10 most common InterPRO features?


In [38]:
ddf5 = ddf.InterPro_accession
ddf5 = ddf5[ddf5 != '-']
# ddf5.head()
ddf5.value_counts().compute().head(10)

InterPro_accession
IPR027417    258
IPR002347    116
IPR003439    104
IPR000182     92
IPR036388     84
IPR036259     80
IPR001789     66
IPR003593     65
IPR029063     62
IPR000515     60
Name: count, dtype: Int64

# 6. If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?

In [62]:
ddf6 = ddf[['InterPro_accession', 'Sequence_length', 'Start_location', 'Stop_location']].compute()
ddf6 =ddf6[ddf6.InterPro_accession != '-']
ddf6 = ddf6[ddf6.Sequence_length * 0.9 <  (ddf6.Stop_location - ddf6.Start_location)]
ddf6_result = ddf6.InterPro_accession.value_counts()[:10]
ddf6_result

InterPro_accession
IPR036259    51
IPR027417    44
IPR000182    32
IPR029068    30
IPR020846    26
IPR016181    24
IPR029058    23
IPR036291    23
IPR036388    22
IPR029063    19
Name: count, dtype: Int64

# 7. If you look at those features which also have textual annotation, what is the top 10 most common word found in that annotation?

In [75]:
ddf6_result.values.sum()

294

In [97]:
ddf7 = ddf[['InterPro_accession', 'InterPro_description', 'Sequence_length', 'Start_location', 'Stop_location']].compute()
ddf7 =ddf7[ddf7.InterPro_accession != '-']
ddf7 = ddf7[ddf7.Sequence_length * 0.9 <  (ddf7.Stop_location - ddf7.Start_location)]
words = ddf7.InterPro_description.str.split(' ').sum()
test = Counter(words)
most_common = sorted(test.items(), key=lambda x:x[1], reverse=True)[:10]
most_common

[('superfamily', 584),
 ('protein', 506),
 ('domain', 451),
 ('family', 126),
 ('transporter', 110),
 ('Ribosomal', 107),
 ('subunit', 98),
 ('hydrolase', 93),
 ('synthase', 86),
 ('Protein', 81)]

# 8. And the top 10 least common?

In [98]:
least_common = sorted(test.items(), key=lambda x:x[1], reverse=False)[:10]
least_common

[('R', 1),
 ('Peptidyl-arginine', 1),
 ('deiminase,', 1),
 ('Porphyromonas-type', 1),
 ('Nucleotide-binding', 1),
 ('plait', 1),
 ('L23/L15e', 1),
 ('beta-ketoacyl', 1),
 ("3'-terminal", 1),
 ('cyclase/enolpyruvate', 1)]

# 9. Combining your answers for Q6 and Q7, what are the 10 most commons words found for the largest InterPRO features?

In [134]:
ddf9 = ddf[['InterPro_accession', 'InterPro_description', 'Sequence_length', 'Start_location', 'Stop_location']].compute()
ddf9 = ddf9[ddf9.InterPro_accession != '-']
ddf9 =  ddf9[ddf9.Sequence_length * 0.9 <  (ddf9.Stop_location - ddf9.Start_location)]

ddf9 = ddf9[['InterPro_accession', 'InterPro_description', 'Sequence_length']].sort_values(by='Sequence_length', ascending=False)
biggest = ddf9.drop_duplicates(subset=['InterPro_accession']).InterPro_accession[:10]

ddf9_words = ddf9[ddf9['InterPro_accession'].isin(biggest)]
words = ddf9_words.InterPro_description.str.split(' ').sum()
test = Counter(words)
most_common = sorted(test.items(), key=lambda x:x[1], reverse=True)[:10]
most_common

[('subunit', 11),
 ('DNA', 7),
 ('DNA-directed', 6),
 ('RNA', 6),
 ('helicase', 5),
 ('polymerase,', 5),
 ('type', 4),
 ('AddA', 3),
 ('helicase,', 2),
 ('UvrD/REP', 2)]

# 10. What is the coefficient of correlation () between the size of the protein and the number of features found?

3209

In [174]:
import numpy as np
# np.corrcoef(ddf.Protein_accession, ddf.Sequence_length)
test = ddf[['Sequence_length', 'Start_location', 'Stop_location']].compute()
features = len(ddf.InterPro_accession.unique())
test['features'] = test.Stop_location - test.Start_location
n = len(test)

sumX = np.sum(test.Sequence_length)
sumY = np.sum(features)
sumXY = np.sum(test.Sequence_length * features)
sumXproduct = np.sum(test.Sequence_length**2)
sumYproduct = np.sum(features**2)
r = (n * sumXY  - (sumX * sumY))
test = np.sqrt(n * sumXproduct - sumX**2) * (n * sumYproduct - sumY**2)
(r/test)

0.0003371265504680764